In [ ]:
# Bag file is input, from realsense viewer

# Script 1 extracts rgb frames from bag
# Script 2 extract depth images/data from bag
# Script 3 flips all images in a given folder

In [5]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os

#script to extract all rgb frames from bag file

# ==== CONFIGURATION ====
bag_file = "/home/aaronmcafee/Documents/exp1z.bag"        # Change this to your .bag path
output_dir = "/home/aaronmcafee/Documents/physVid/exp1/rgbz"               # Change this to your target folder
os.makedirs(output_dir, exist_ok=True)
# ========================

# Set up pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_device_from_file(bag_file, repeat_playback=False)

# Start stream
profile = pipeline.start(config)

# Access playback object to control playback
device = profile.get_device()
playback = device.as_playback()
playback.set_real_time(False)  # Ensures processing is as fast as possible

# Frame counter
frame_count = 0

try:
    while True:
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()

        if not color_frame:
            continue

        # Convert frame to numpy array
        color_image = np.asanyarray(color_frame.get_data())

        # Save as image
        filename = os.path.join(output_dir, f"v2_{frame_count:06d}.png")
        color_image_rgb = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
        cv2.imwrite(filename, color_image_rgb)
        print(f"Saved: {filename}")

        frame_count += 1

except RuntimeError:
    print(f"Finished. Total frames saved: {frame_count}")

pipeline.stop()


Saved: /home/aaronmcafee/Documents/physVid/exp1/rgbz/v2_000000.png
Saved: /home/aaronmcafee/Documents/physVid/exp1/rgbz/v2_000001.png
Saved: /home/aaronmcafee/Documents/physVid/exp1/rgbz/v2_000002.png
Saved: /home/aaronmcafee/Documents/physVid/exp1/rgbz/v2_000003.png
Saved: /home/aaronmcafee/Documents/physVid/exp1/rgbz/v2_000004.png
Saved: /home/aaronmcafee/Documents/physVid/exp1/rgbz/v2_000005.png
Saved: /home/aaronmcafee/Documents/physVid/exp1/rgbz/v2_000006.png
Saved: /home/aaronmcafee/Documents/physVid/exp1/rgbz/v2_000007.png
Saved: /home/aaronmcafee/Documents/physVid/exp1/rgbz/v2_000008.png
Saved: /home/aaronmcafee/Documents/physVid/exp1/rgbz/v2_000009.png
Saved: /home/aaronmcafee/Documents/physVid/exp1/rgbz/v2_000010.png
Saved: /home/aaronmcafee/Documents/physVid/exp1/rgbz/v2_000011.png
Saved: /home/aaronmcafee/Documents/physVid/exp1/rgbz/v2_000012.png
Saved: /home/aaronmcafee/Documents/physVid/exp1/rgbz/v2_000013.png
Saved: /home/aaronmcafee/Documents/physVid/exp1/rgbz/v2_000014

In [8]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os
import json

#script to extract depth frames, and metadata from bag files

# ==== CONFIGURATION ====
bag_file = "/home/aaronmcafee/Documents/exp1y.bag"        # Change this to your .bag path
output_dir = "/home/aaronmcafee/Documents/physVid/exp1/depthy"               # Change this to your target folder
os.makedirs(output_dir, exist_ok=True)
# ========================

# Set up pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_device_from_file(bag_file, repeat_playback=False)

# Start stream
profile = pipeline.start(config)

# Access playback object
device = profile.get_device()
playback = device.as_playback()
playback.set_real_time(False)

# Get depth intrinsics (only once needed)
depth_stream = profile.get_stream(rs.stream.depth)
depth_intrinsics = depth_stream.as_video_stream_profile().get_intrinsics()

frame_count = 0

try:
    while True:
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()

        if not depth_frame:
            continue

        # Convert depth to NumPy array
        depth_image = np.asanyarray(depth_frame.get_data())

        # Save raw depth as .npy
        npy_filename = os.path.join(output_dir, f"depth_{frame_count:06d}.npy")
        np.save(npy_filename, depth_image)

        # Save depth as 16-bit PNG
        png_filename = os.path.join(output_dir, f"depth_{frame_count:06d}.png")
        cv2.imwrite(png_filename, depth_image)

        # Save metadata
        metadata = {
            "frame_number": depth_frame.get_frame_number(),
            "timestamp": depth_frame.get_timestamp(),
            "width": depth_image.shape[1],
            "height": depth_image.shape[0],
            "intrinsics": {
                "width": depth_intrinsics.width,
                "height": depth_intrinsics.height,
                "ppx": depth_intrinsics.ppx,
                "ppy": depth_intrinsics.ppy,
                "fx": depth_intrinsics.fx,
                "fy": depth_intrinsics.fy,
                "model": str(depth_intrinsics.model),
                "coeffs": depth_intrinsics.coeffs
            }
        }
        meta_filename = os.path.join(output_dir, f"depth_{frame_count:06d}_meta.json")
        with open(meta_filename, 'w') as f:
            json.dump(metadata, f, indent=2)

        print(f"Saved frame {frame_count}: .png, .npy, .json")

        frame_count += 1

except RuntimeError:
    print(f"Finished. Total frames saved: {frame_count}")

pipeline.stop()

Saved frame 0: .png, .npy, .json
Saved frame 1: .png, .npy, .json
Saved frame 2: .png, .npy, .json
Saved frame 3: .png, .npy, .json
Saved frame 4: .png, .npy, .json
Saved frame 5: .png, .npy, .json
Saved frame 6: .png, .npy, .json
Saved frame 7: .png, .npy, .json
Saved frame 8: .png, .npy, .json
Saved frame 9: .png, .npy, .json
Saved frame 10: .png, .npy, .json
Saved frame 11: .png, .npy, .json
Saved frame 12: .png, .npy, .json
Saved frame 13: .png, .npy, .json
Saved frame 14: .png, .npy, .json
Saved frame 15: .png, .npy, .json
Saved frame 16: .png, .npy, .json
Saved frame 17: .png, .npy, .json
Saved frame 18: .png, .npy, .json
Saved frame 19: .png, .npy, .json
Saved frame 20: .png, .npy, .json
Saved frame 21: .png, .npy, .json
Saved frame 22: .png, .npy, .json
Saved frame 23: .png, .npy, .json
Saved frame 24: .png, .npy, .json
Saved frame 25: .png, .npy, .json
Saved frame 26: .png, .npy, .json
Saved frame 27: .png, .npy, .json
Saved frame 28: .png, .npy, .json
Saved frame 29: .png, .n

In [35]:
#Script to run image flips on selected folder

from PIL import Image
import os

input_folder = "/home/aaronmcafee/Documents/bigVid/filter"  # Change this
for filename in os.listdir(input_folder):
    if filename.lower().endswith('.png'):
        img_path = os.path.join(input_folder, filename)
        img = Image.open(img_path)
        flipped = img.transpose(Image.FLIP_TOP_BOTTOM)
        flipped.save(img_path)  # Overwrite original
